In [1]:
import pandas as pd
import sys,math
from collections import Counter
import numpy as np
import random



In [2]:
file = pd.read_csv('IMDB Dataset.csv')
raw_reviews = file['review'].to_list()
raw_labels = file['sentiment'].to_list()

In [3]:
tokens = list(map(lambda x:set(x.split(" ")), raw_reviews))

In [4]:
wordcnt = Counter()
for sent in tokens:
    i=0
    for word in sent:
        wordcnt[word] -= 1

In [5]:
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

In [6]:
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

In [7]:
concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])

        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)

In [8]:
alpha, iterations = (0.05, 2)
hidden_size, window, negative = (50,2,5)

In [9]:
weights_0_1 = (np.random.rand(len(vocab), hidden_size) - 0.5)*0.2
weights_1_2 = np.random.rand(len(vocab), hidden_size)*0

In [10]:
#np.mean(weights_0_1[[2,5000]], axis=0)

In [11]:
layer_2_target = np.zeros(negative + 1)
layer_2_target[0] = 1

In [12]:
def similar(target="beautiful"):
    target_index = word2index[target]

    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference**2
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [13]:
input_dataset[0][1]

304455

In [14]:
print([[2,3],[3,4]]*2)

[[2, 3], [3, 4], [2, 3], [3, 4]]


In [15]:
for rev_i, review in enumerate(input_dataset*iterations):
    for target_i in range(len(review)):
        target_samples = [review[target_i]] + list(concatenated[(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])
        
        left_context = review[max(0,target_i - window): target_i]
        
        right_context = review[target_i+ 1: min(len(review), target_i+window)]

        layer_1 = np.mean(weights_0_1[left_context+right_context], axis=0)
        layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
        layer_2_delta = layer_2 - layer_2_target
        layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

        weights_0_1[left_context+right_context] -= layer_1_delta*alpha
        weights_1_2[target_samples] -= np.outer(layer_2_delta, layer_1)*alpha
    if (rev_i%500 == 0):
         sys.stdout.write('\rProgress:  '+ str(rev_i/float(len(input_dataset)*iterations)) + ' ' + str(similar('terrible')))
         sys.stdout.write('\rProgress: '+ str(rev_i/float(len(input_dataset)*iterations)))
print("")        
print(f"Answer: {similar('terrible')}")
       
   
        

    

Progress: 0.9955 [('terrible', -0.0), ('IS', -3.239742908875227), ('Why', -3.3158155914355394), ('Mel', -3.3231003596491866), ('small', -3.3235185250897312), ('fantasy', -3.3445382818994247), ('production,', -3.3815141050188418), ("would've", -3.4646347376558895), ('Friday', -3.4909255663519727), ('fast', -3.537610492020584)])]956)]208420493420245)]
Answer: [('terrible', -0.0), ('IS', -3.243674572102079), ('fantasy', -3.2583295560895733), ('Mel', -3.345444530088766), ("would've", -3.3574828743416583), ('small', -3.360578787748396), ('Why', -3.3653191772127298), ('fast', -3.3897972697187044), ('production,', -3.4108757562782843), ('Friday', -3.420316421565572)]


In [16]:
norms = np.sum(weights_0_1**2, axis=1)
norms.resize(norms.shape[0],1)
normed_weights = weights_0_1*norms

In [17]:
def make_sent_vect(words):
    indices = list(map(lambda x:word2index[x], filter(lambda x:x in word2index, words)))
    return np.mean(normed_weights[indices], axis=0)

In [18]:
review2vectors = list()
for review in tokens:
    review2vectors.append(make_sent_vect(review))
review2vectors = np.array(review2vectors)

In [19]:
def most_similar_reviews(review):
    v = make_sent_vect(review)
    scores = Counter()
    for i,val in enumerate(review2vectors.dot(v)):
        scores[i] = val
    most_similar = list()
    for idx,score in scores.most_common(3):
        most_similar.append(raw_reviews[idx][0:40])
    return most_similar    

In [20]:
most_similar_reviews(['boring','worst'])

['My favorite movie. What a great story th',
 'As with most Rosalind Russell movies, th',
 'A great film in its genre, the direction']

In [21]:
import sys,random,math
from collections import Counter
import numpy as np


In [22]:
f = open('tasksv11/en/qa1_single-supporting-fact_train.txt','r')
raw = f.readlines()
f.close()

In [23]:
tokens = list()
for line in raw[0:1000]:
    tokens.append(line.lower().replace("\n","").replace(".","").split(" ")[1:])


In [24]:
print(tokens[0:3])

[['mary', 'moved', 'to', 'the', 'bathroom'], ['john', 'went', 'to', 'the', 'hallway'], ['where', 'is', 'mary?', '\tbathroom\t1']]


In [25]:
vocab = set()
for sent in tokens:
    for word in sent:
        vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

In [26]:
def words2indices(sentence):
    idx = list()
    for word in sentence:
        idx.append(word2index[word])
    return idx

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x/e_x.sum(axis=0)

In [27]:
np.random.seed(1)

In [28]:
embed_size = 10

embed = (np.random.rand(len(vocab), embed_size) - 0.5)*0.1

recurrent = np.eye(embed_size)

start = np.zeros(embed_size)

decoder = (np.random.rand(embed_size, len(vocab)) - 0.5)*0.1

one_hot = np.eye(len(vocab))

In [29]:
def predict(sent):

    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)

    loss = 0

    preds = list()
    for target_i in range(len(sent)):
        layer = {}

        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))
        loss += -np.log(layer['pred'][sent[target_i]])
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[sent[target_i]]
        layers.append(layer)

    return layers, loss

In [30]:
for iter in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])
    layers, loss = predict(sent)

    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx]
        target = sent[layer_idx - 1]

        if(layer_idx > 0):
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            if(layer_idx == len(layers) - 1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx + 1]['hidden_delta'].dot(recurrent.transpose( ))
        else:  
            layer['hidden_delta'] = layers[layer_idx + 1]['hidden_delta'].dot(recurrent.transpose())

    start -= layers[0]['hidden_delta']*alpha/float(len(sent))
    for layer_idx,layer in enumerate(layers[1:]):

        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta'])*alpha/float(len(sent))

        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta']*alpha/float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta'])*alpha/float(len(sent))

    if(iter % 1000 ==0):
        print("Perplexity: "+ str(np.exp(loss/len(sent))))